Welcome to exercise two of week three of “Apache Spark for Scalable Machine Learning on BigData”. In this exercise we’ll work on clustering.

Let’s create our DataFrame again:

In [2]:
%sh

# delete files from previous runs
#rm -f hmp.parquet*

# download the file containing the data in PARQUET format
#wget https://github.com/IBM/coursera/raw/master/hmp.parquet
#ls -ltr /databricks/driver
#mkdir /dbfs/tmp/HMPPARQ/
#cp /databricks/driver/hmp.parquet /dbfs/tmp/HMPPARQ/
ls -ltr /dbfs/tmp/HMPPARQ/

total 0
-rw-r--r-- 1 root root 932997 Jun 25 12:08 hmp.parquet

In [3]:
# create a dataframe out of it
df = spark.read.parquet('/tmp/HMPPARQ/hmp.parquet')

# register a corresponding query table
df.createOrReplaceTempView('df')
df.show()

+---+---+---+--------------------+-----------+
 x| y| z| source| class|
+---+---+---+--------------------+-----------+
 22| 49| 35|Accelerometer-201...|Brush_teeth|
 22| 49| 35|Accelerometer-201...|Brush_teeth|
 22| 52| 35|Accelerometer-201...|Brush_teeth|
 22| 52| 35|Accelerometer-201...|Brush_teeth|
 21| 52| 34|Accelerometer-201...|Brush_teeth|
 22| 51| 34|Accelerometer-201...|Brush_teeth|
 20| 50| 35|Accelerometer-201...|Brush_teeth|
 22| 52| 34|Accelerometer-201...|Brush_teeth|
 22| 50| 34|Accelerometer-201...|Brush_teeth|
 22| 51| 35|Accelerometer-201...|Brush_teeth|
 21| 51| 33|Accelerometer-201...|Brush_teeth|
 20| 50| 34|Accelerometer-201...|Brush_teeth|
 21| 49| 33|Accelerometer-201...|Brush_teeth|
 21| 49| 33|Accelerometer-201...|Brush_teeth|
 20| 51| 35|Accelerometer-201...|Brush_teeth|
 18| 49| 34|Accelerometer-201...|Brush_teeth|
 19| 48| 34|Accelerometer-201...|Brush_teeth|
 16| 53| 34|Accelerometer-201...|Brush_teeth|
 18| 52| 35|Accelerometer-201...|Brush_teeth|
 18| 51| 32|Accelerometer-201...|Brush_teeth|
+---+---+---+--------------------+-----------+
only showing top 20 rows

Let’s reuse our feature engineering pipeline.

In [5]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline

indexer = StringIndexer(inputCol="class", outputCol="classIndex")
encoder = OneHotEncoder(inputCol="classIndex", outputCol="categoryVec")
vectorAssembler = VectorAssembler(inputCols=["x","y","z"],
                                  outputCol="features")
normalizer = Normalizer(inputCol="features", outputCol="features_norm", p=1.0)

pipeline = Pipeline(stages=[indexer, encoder, vectorAssembler, normalizer])
model = pipeline.fit(df)
prediction = model.transform(df)
prediction.show()

+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
 x| y| z| source| class|classIndex| categoryVec| features| features_norm|
+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
 22| 49| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.20754716981132...|
 22| 49| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.20754716981132...|
 22| 52| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.20183486238532...|
 22| 52| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.20183486238532...|
 21| 52| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|[0.19626168224299...|
 22| 51| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|[0.20560747663551...|
 20| 50| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[20.0,50.0,35.0]|[0.19047619047619...|
 22| 52| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,52.0,34.0]|[0.20370370370370...|
 22| 50| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,50.0,34.0]|[0.20754716981132...|
 22| 51| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[22.0,51.0,35.0]|[0.20370370370370...|
 21| 51| 33|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[21.0,51.0,33.0]|[0.2,0.4857142857...|
 20| 50| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[20.0,50.0,34.0]|[0.19230769230769...|
 21| 49| 33|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[21.0,49.0,33.0]|[0.20388349514563...|
 21| 49| 33|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[21.0,49.0,33.0]|[0.20388349514563...|
 20| 51| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[20.0,51.0,35.0]|[0.18867924528301...|
 18| 49| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[18.0,49.0,34.0]|[0.17821782178217...|
 19| 48| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[19.0,48.0,34.0]|[0.18811881188118...|
 16| 53| 34|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[16.0,53.0,34.0]|[0.15533980582524...|
 18| 52| 35|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[18.0,52.0,35.0]|[0.17142857142857...|
 18| 51| 32|Accelerometer-201...|Brush_teeth| 6.0|(13,[6],[1.0])|[18.0,51.0,32.0]|[0.17821782178217...|
+---+---+---+--------------------+-----------+----------+--------------+----------------+--------------------+
only showing top 20 rows

Now let’s create a new pipeline for kmeans.

In [7]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans(featuresCol="features").setK(14).setSeed(1)
pipeline = Pipeline(stages=[vectorAssembler, kmeans])
model = pipeline.fit(df)
predictions = model.transform(df)

evaluator = ClusteringEvaluator()

silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.41244594513295846

We have 14 different movement patterns in the dataset, so setting K of KMeans to 14 is a good idea. But please experiment with different values for K, do you find a sweet spot? The closer Silhouette gets to 1, the better.

https://en.wikipedia.org/wiki/Silhouette_(clustering)

In [9]:
# please change the pipeline the check performance for different K, feel free to use a loop

Now please extend the pipeline to work on the normalized features. You need to tell KMeans to use the normalized feature column and change the pipeline in order to contain the normalizer stage as well.